In [212]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import xarray as xr
import hvplot.xarray  # noqa


In [213]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [214]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [215]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

In [216]:
df_market_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
dtypes: float64(7)
memory usage: 2.6+ KB


### Prepare the Data

In [217]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = (StandardScaler().fit_transform(df_market_data))
scaled_data[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [218]:
# Create a DataFrame with the scaled data
df_crypto_scaled = pd.DataFrame(
    scaled_data,
    columns=["price_change_percentage_24h", 
    "price_change_percentage_7d",
    "price_change_percentage_14d",
    "price_change_percentage_30d", 
    "price_change_percentage_60d",
    "price_change_percentage_200d", 
    "price_change_percentage_1y"]
)

# Copy the crypto names from the original data

# Set the coinid column as index
df_crypto_scaled["coin_id"] = df_market_data.index

df_crypto_scaled = df_crypto_scaled.set_index("coin_id")

# Display sample data
df_crypto_scaled.head(7)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284


---

### Find the Best Value for k Using the Original Data.

In [219]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [220]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1, n_init=10)
    k_model.fit(df_crypto_scaled)
    inertia.append(k_model.inertia_)

c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is k

In [221]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create a DataFrame with the data to plot the Elbow curve
df_elbow.head(7)

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,63.858668
5,6,53.057788
6,7,44.406791


In [222]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow1 = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="The Curve", 
    xticks=k
)
elbow1

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Best value for k = 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [223]:
# Initialize the K-Means model using the best value for k
model4 = KMeans(n_clusters=4)

In [224]:
# Fit the K-Means model using the scaled data
model4.fit(df_crypto_scaled)

c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [225]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model4.predict(df_crypto_scaled)

# Print the resulting array of cluster values.
crypto_clusters

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 0, 1, 1, 2, 1, 1, 1, 1])

In [226]:
# Create a copy of the DataFrame
df_crypto_predict = df_crypto_scaled.copy()

In [227]:
# Add a new column to the DataFrame with the predicted clusters
df_crypto_predict["Predicted Clusters"] = crypto_clusters

# Display sample data
df_crypto_predict.head(7)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted Clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,0
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,0


In [228]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_original = df_crypto_predict.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Predicted Clusters",
    hover_cols="coin_id"
)
crypto_original

:NdOverlay   [Predicted Clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [229]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [230]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca = pca.fit_transform(df_crypto_predict)
# View the first five rows of the DataFrame. 
crypto_pca[:5]

array([[-0.98927075,  0.71515999,  0.49382193],
       [-0.80114992,  0.38686277,  1.01001186],
       [-0.25723805, -0.29436771, -0.71543698],
       [-0.28931239, -0.35459601, -0.56121306],
       [-1.84988759,  1.75473268,  1.71168922]])

In [231]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.38141983, 0.33155857, 0.17643803])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 89% of the three principal components.

In [232]:
# Create a new DataFrame with the PCA data.
crypto_pca_df = pd.DataFrame(
    crypto_pca,
    columns=["PCA1", "PCA2", "coin_id"]
)
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
crypto_pca_df["coin_id"] =df_market_data.index

# Set the coinid column as index
crypto_pca_df = crypto_pca_df.set_index("coin_id")

# Display sample data
crypto_pca_df.head(7)

,PCA1,PCA2
coin_id,,
bitcoin,-0.989271,0.715160
ethereum,-0.801150,0.386863
tether,-0.257238,-0.294368
ripple,-0.289312,-0.354596
bitcoin-cash,-1.849888,1.754733
binancecoin,-1.052689,1.268868
chainlink,-0.914743,0.474010


---

### Find the Best Value for k Using the PCA Data

In [233]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [234]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0, n_init=10)
    model.fit(crypto_pca_df)
    inertia.append(model.inertia_)

c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is k

In [235]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k_PCA": k,
    "inertia_PCA": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [236]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(
    x="k_PCA", 
    y="inertia_PCA", 
    title="Curve Using PCA Data", 
    xticks=k
    )
elbow_plot_pca

:Curve   [k_PCA]   (inertia_PCA)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** best value for 'k' with PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it's not different than the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [237]:
# Initialize the K-Means model using the best value for k
model4pca = KMeans(n_clusters=4, random_state= 1,n_init=10)

In [238]:
# Fit the K-Means model using the PCA data
model4pca.fit(crypto_pca_df)

c:\Users\jnnfr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, n_init=10, random_state=1)

In [239]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_clusters_pca = model4pca.predict(crypto_pca_df)
# Print the resulting array of cluster values.
crypto_clusters_pca

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 3, 0, 0, 1, 0, 0, 0, 0])

In [240]:
# Create a copy of the DataFrame with the PCA data
copy_crypto_clusters_pca = crypto_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
copy_crypto_clusters_pca["Predicted Cluster"] = crypto_clusters_pca

# Display sample data
copy_crypto_clusters_pca.head(7)


,PCA1,PCA2,Predicted Cluster
coin_id,,,
bitcoin,-0.989271,0.715160,3
ethereum,-0.801150,0.386863,3
tether,-0.257238,-0.294368,0
ripple,-0.289312,-0.354596,0
bitcoin-cash,-1.849888,1.754733,3
binancecoin,-1.052689,1.268868,3
chainlink,-0.914743,0.474010,3


In [241]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
cryptoPCA = copy_crypto_clusters_pca.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="Predicted Cluster",
    hover_cols="coin_id"
)
cryptoPCA

:NdOverlay   [Predicted Cluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [246]:
# Composite plot to contrast the Elbow curves
#INCOMPLETE

elbow1 + elbow_plot_pca

# elbow1 + elbow_plot_pca.hvplot(
#     x=["k", "k_PCA"],
#     y=["inertia", "inertia_PCA"]) 


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k_PCA]   (inertia_PCA)

In [ ]:
# Composite plot to contrast the clusters
#INCOMPLETE
crypto_original + cryptoPCA

:Layout
   .NdOverlay.I  :NdOverlay   [Predicted Clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Predicted Cluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 